In [26]:
import numpy as np
import pandas as pd
import scanpy as sc

In [27]:
adata = sc.read("lyon_transcriptomics_scanpy_normalised.h5")
adata.obs["Metadata_treatments"] = adata.obs["Metadata_treatments"].astype(str)
adata.obs["Metadata_target"] = adata.obs["Metadata_target"].astype(str)
adata.obs["Metadata_Mode_of_Action"] = adata.obs["Metadata_Mode_of_Action"].astype(str)
adata.obs.loc[adata.obs["Metadata_treatments"] == "DMSO", "Metadata_target"] = "DMSO"
adata.obs.loc[adata.obs["Metadata_treatments"] == "DMSO", "Metadata_Mode_of_Action"] = "DMSO"

In [28]:
# rename_dict = {
#     "cell wal" : 'Cell wall',
#     "lipid other": "lipid",
#     "lipid SBI": "lipid",
# }
# adata.obs["Metadata_Mode_of_Action"].replace(rename_dict, inplace=True)
# adata.obs["Metadata_Mode_of_Action"] = adata.obs["Metadata_Mode_of_Action"].str.capitalize()

# adata = adata[adata.obs["Metadata_Mode_of_Action"].isin(adata.obs["Metadata_Mode_of_Action"].value_counts()[adata.obs["Metadata_Mode_of_Action"].value_counts() >= 10].index)]
adata.obs["Metadata_Mode_of_Action"].value_counts()

Metadata_Mode_of_Action
lipid SBI           245
respiration         178
lipid other         129
GPI                 102
cytoskeleton         94
amino acid           72
prot metabolism      58
proteins             57
DMSO                 56
DHODH                56
cell wal             44
HOG                  43
Cell wall            32
Multi                30
HDAC                 28
ATP proton pump      25
GPI                  25
IMPDH                25
Name: count, dtype: int64

In [29]:
import pickle

gene_mapping = pickle.load(open("gene_mapping_botrytis.pkl", "rb"))

In [30]:
pickle.dump(gene_mapping, open("gene_mapping_botrytis.pkl", "wb"))

In [31]:
adata.var["Gene Name"] = [gene_mapping[x] if x in gene_mapping else x for x in adata.var.index]

In [32]:
adata.var["Gene Name"] = adata.var["Gene Name"].astype(str)
# Check for duplicate gene names and add a suffix if necessary
gene_name_counts = adata.var["Gene Name"].value_counts()
duplicate_gene_names = gene_name_counts[gene_name_counts > 1].index

for gene_name in duplicate_gene_names:
    indices = adata.var[adata.var["Gene Name"] == gene_name].index
    for i, idx in enumerate(indices):
        adata.var.at[idx, "Gene Name"] = f"{gene_name}_{i + 1}"

In [33]:
adata.obs["Metadata_Mode_of_Action"].unique()

array(['DMSO', 'lipid other ', 'lipid SBI ', 'prot metabolism ',
       'respiration ', 'IMPDH', 'cell wal ', 'Cell wall ',
       'cytoskeleton ', 'DHODH', 'GPI', 'HDAC', 'HOG', 'amino acid ',
       'ATP proton pump ', 'GPI ', 'proteins ', 'Multi'], dtype=object)

In [34]:
def gene_expression_per_group(subsample_adata, group, reference, genes2display=5):
    sc.tl.rank_genes_groups(
        subsample_adata,
        group,
        reference=reference,
        # method="wilcoxon",
    )
    # sc.pl.rank_genes_groups_dotplot(
    #     subsample_adata,
    #     n_genes = 5,
    #     standard_scale='var',
    #     gene_symbols = "Gene Name"
    # )
    # sc.pl.rank_genes_groups_dotplot(
    #     adata,
    #     n_genes = 5,
    #     standard_scale='var',
    #     gene_symbols = "Summary functional categories"
    # )
    subsample_adata = subsample_adata[subsample_adata.obs[group] != reference]

    subsample_adata.obs[group] = subsample_adata.obs[group].astype("category")
    sc.tl.dendrogram(subsample_adata, groupby=group)

    sc.pl.rank_genes_groups_dotplot(
        subsample_adata,
        n_genes=genes2display,
        # standard_scale='var',
        vmin=-3,
        vmax=3,
        cmap="bwr",
        values_to_plot="logfoldchanges",
        gene_symbols="Gene Name",
    )
    # sc.pl.rank_genes_groups_heatmap(
    #     subsample_adata,
    #     n_genes=10,
    #     use_raw=False,
    #     swap_axes=True,
    #     show_gene_labels=False,
    #     vmin=-3,
    #     vmax=3,
    #     cmap="bwr",
    #     gene_symbols = "Gene Name"
    # )
    # sc.pl.rank_genes_groups_dotplot(
    #     adata,
    #     n_genes=5,  # Adjust to the number of genes you want to plot
    #     #cmap = "bwr",
    #     values_to_plot="logfoldchanges",
    #     gene_symbols = "Description summary"
    # )

    # sc.pl.rank_genes_groups(subsample_adata, n_genes=10, sharey=False, gene_symbols = "Gene Name")

    # for member in subsample_adata.obs[group].unique():
    #     if member == "DMSO":
    #         continue
    #     sc.pl.rank_genes_groups_violin(subsample_adata, groups=member, n_genes=5, gene_symbols = "Gene Name")

In [35]:
# gene_expression_per_group(adata, "Metadata_target", "DMSO", 2)

In [36]:
# for moa in adata.obs["Metadata_Mode_of_Action"].unique():
#     if moa == "DMSO":
#         continue
#     adata_small = adata[adata.obs["Metadata_Mode_of_Action"].isin([moa, "DMSO"])].copy()
#     if adata_small.obs["Metadata_target"].nunique() < 2:
#         continue
#     print(moa)
#     adata_small.obs.loc[adata_small.obs["Metadata_Mode_of_Action"] == "DMSO", "Metadata_target"] = "DMSO"
#     print(adata_small.obs["Metadata_target"].value_counts())
#     gene_expression_per_group(adata_small, "Metadata_target", "DMSO")

In [37]:
df = pd.read_csv("full_raw_matrix_and_meta.csv", index_col=0)
leiden_dict = pickle.load(open("leiden_dict_lyon.pkl", "rb"))
df_meta = pd.read_csv("transcriptomics_moa_target 1.csv", index_col=0)
df_meta.loc[df_meta["Pathway / Target"] == "DMSO", "MoA_tr"] = "DMSO"
df_meta.loc[df_meta["Pathway / Target"] == "DMSO", "MoA_tr"] = "DMSO"
# df_meta.columns = [ "Metadata_" + x for x in df_meta.columns]
moa_dict = dict(zip(df_meta["BCS-code"], df_meta["MoA_tr"]))
target_dict = dict(zip(df_meta["BCS-code"], df_meta["MoA_tr"]))
moa_dict["DMSO"] = "DMSO"
target_dict["DMSO"] = "DMSO"

df["Metadata_MoA"] = np.nan
df["Metadata_target"] = np.nan
df["Metadata_MOA"] = df["Metadata_treatments"].map(moa_dict)
df["Metadata_target"] = df["Metadata_treatments"].map(target_dict)
df["Metadata_leiden"] = df["Metadata_treatments"].map(leiden_dict)

gene_mapping = pickle.load(open("gene_mapping_botrytis.pkl", "rb"))
print(df.shape)
df.rename(
    columns={
        "Metadata_MOA": "Metadata_Mode_of_Action",
        "Metadata_treatments": "Metadata_treatments",
    },
    inplace=True,
)
df.loc[df["Metadata_treatments"] == "DMSO", "Metadata_concentration"] = 1

(1440, 12205)


In [38]:
df["Metadata_leiden"].unique()

array(['Leiden cluster 3', 'Leiden cluster 6', 'Leiden cluster 5',
       'Leiden cluster 10', 'Leiden cluster 20', 'Leiden cluster 15',
       'Leiden cluster 0', 'Leiden cluster 8', 'Leiden cluster 13',
       'Leiden cluster 17', 'Leiden cluster 1', 'Leiden cluster 7',
       'Leiden cluster 12', 'Leiden cluster 14', 'Leiden cluster 16',
       'Leiden cluster 2', 'Leiden cluster 19', 'Leiden cluster 18',
       'Leiden cluster 11', 'Leiden cluster 4', 'Leiden cluster 9', nan],
      dtype=object)

In [39]:
s = df.columns.to_series().groupby(df.columns)

df.columns = np.where(
    s.transform("size") > 1, df.columns + s.cumcount().add(1).astype(str), df.columns
)

In [40]:
gene_name_counts = df.columns.value_counts()
gene_name_counts[gene_name_counts > 1]

Series([], Name: count, dtype: int64)

In [41]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats


def deseq(df: pd.DataFrame, group: str, reference: str) -> pd.DataFrame:
    """
    Perform differential expression analysis using DESeq2.

    This function takes a DataFrame containing gene expression counts and metadata,
    and performs differential expression analysis using the DESeq2 method. It prepares
    the data, fits the DESeq2 model, and returns a summary of the results sorted by
    adjusted p-values.

    Parameters:
    ----------
    df : pandas.DataFrame
        A DataFrame where rows represent genes and columns represent samples. The DataFrame
        should include metadata columns prefixed with "Metadata_" and a column for the
        grouping variable.

    group : str
        The name of the column in the DataFrame that specifies the grouping variable for
        the differential expression analysis. This variable should contain the treatment
        groups to be compared.

    reference : str
        The reference level for the contrast in the differential expression analysis. This
        should be one of the levels present in the `group` column.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the results of the differential expression analysis, sorted
        by adjusted p-values.
    """

    # if os.path.isfile(f"differential_expression_tables/{treatment}_differential_expression.csv"):
    #     print("File exists, skipping")
    #     return
    # Identify gene columns by excluding metadata columns
    gene_columns = df.columns[~df.columns.str.startswith("Metadata_")]

    # Drop rows with NA values in specified columns
    df.dropna(subset=["Metadata_plate", group], inplace=True)

    # Prepare counts DataFrame with genes as rows and samples as columns, filling NA with 0
    counts = df[gene_columns].fillna(0).astype(int)

    # Retain only genes with non-zero variance
    counts = counts.loc[:, counts.var() != 0]

    # Prepare metadata DataFrame
    metadata = df[["Metadata_plate", group]]
    print(counts.shape)  # Print shape of counts DataFrame
    print(metadata.shape)  # Print shape of metadata DataFrame

    # Initialize inference with specified number of CPUs
    inference = DefaultInference(n_cpus=14)

    # Create DESeqDataSet object
    dds = DeseqDataSet(
        counts=counts.reset_index(drop=True),
        metadata=metadata.reset_index(drop=True),
        design=f"~ Metadata_plate + {group}",
        refit_cooks=True,
        inference=inference,
    )

    # Fit size factors and dispersions
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()

    # Refitting outlier counts if specified
    if dds.refit_cooks:
        dds.refit()

    # Set up and run differential expression statistics
    ds = DeseqStats(
        dds,
        contrast=[group, treatment, reference],
        alpha=0.05,
        cooks_filter=True,
        independent_filter=True,
    )
    ds.run_wald_test()

    # Perform independent filtering or p-value adjustment
    if ds.independent_filter:
        ds._independent_filtering()
    else:
        ds._p_value_adjustment()

    ds.summary()  # Print summary of results

    # Sort results by adjusted p-values
    summary_df = ds.results_df.copy()
    summary_df.sort_values("padj", inplace=True)

    return summary_df  # Return sorted results DataFrame

In [42]:
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
import os


def make_volcano(data_frame, treatment):
    """
    Make an interactive volcano plot as an html. Attach meta data to individual genes.

    Parameters
    ----------
    treatment : string
        Name of the treatment

    Returns
    -------
    fig : object
        The figure object of the volcano plot

    pd.Series : object
        Pandas series containing the log2FoldChange. Will be used later for
        colouring in the plot_gene_atlas function
    """
    # if os.path.isfile(f"differential_expression_tables/{treatment}_differential_expression.csv"):
    #     print("File exists, skipping")
    #     return


    # Convert p-values into -Log10p
    data_frame["-Log10P"] = np.log10(data_frame["padj"]) * -1
    data_frame.loc[data_frame["-Log10P"] < 0, "-Log10P"] = 0

    # Filter for significant Up-regulated and Down-regulated genes
    data_frame["Type hit"] = "Under Threshold"
    data_frame.loc[
        (data_frame["log2FoldChange"] > 1) & (data_frame["padj"] < 0.05), "Type hit"
    ] = "Up-regulated over threshold of P < 0.05 \n and log(2) Fold-Change > 1"
    data_frame.loc[
        (data_frame["log2FoldChange"] < -1) & (data_frame["padj"] < 0.05), "Type hit"
    ] = "Down-regulated over threshold of P < 0.05 \n and log(2) Fold-Change > 1"

    # Filter out non-significant genes
    data_frame.sort_values("-Log10P", ascending=False, inplace=True)
    print(data_frame.head())
    data_frame.to_csv(f"{treatment}_differential_expression.csv")

    data_frame.rename(
        columns={
            "log2FoldChange": "Log(2) Fold-Change",
            "-Log10P": "-Log(10) of adjusted P-value",
        },
        inplace=True,
    )

    all_significant = data_frame[
        data_frame["Type hit"] != "Under Threshold"
    ].sort_values("-Log(10) of adjusted P-value", ascending=False)
    filtered_data = all_significant.iloc[:15]

    fig = plt.figure(figsize=(18, 12))

    colour_dict = {
        "Under Threshold": "lightgrey",
        "Up-regulated over threshold of P < 0.05 \n and log(2) Fold-Change > 1": "cornflowerblue",
        "Down-regulated over threshold of P < 0.05 \n and log(2) Fold-Change > 1": "crimson",
    }

    # Make figure
    g = sns.scatterplot(
        data=data_frame,
        x="Log(2) Fold-Change",
        y="-Log(10) of adjusted P-value",
        hue="Type hit",
        alpha=0.6,
        palette=colour_dict,
        legend=True,
        edgecolor="none",
    )

    # Add lines to show thresholds
    ax = plt.gca()
    ax.axhline(y=np.log10(0.05) * -1, lw=2, ls="--", c="grey")
    ax.axvline(x=1, lw=2, ls="--", c="grey")
    ax.axvline(x=-1, lw=2, ls="--", c="grey")
    ax.spines[["right", "top"]].set_visible(False)
    ax.grid(True, alpha=0.2)
    ax.tick_params(axis="both", which="major", labelsize=12)

    texts = []
    for gene, row in filtered_data.iterrows():
        # Check the log2 fold change to determine text position
        if row["Log(2) Fold-Change"] < 1:
            x_offset = 0.05  # Offset to the right
        else:
            x_offset = -0.05  # Offset to the left for better visibility

        texts.append(
            ax.text(
                row["Log(2) Fold-Change"] + x_offset,
                row["-Log(10) of adjusted P-value"] * 1.01,
                f"{gene_mapping[gene]}",
                fontsize=14,
                color="black",
            )
        )

    # Use adjustText to avoid overlapping labels
    adjust_text(
        texts,
        expand=(1.1, 1.1),
        # only_move='y',  # Move only in the y direction
        arrowprops=dict(color="silver", lw=1),
    )

    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.title(f"Volcano plot for {treatment} vs. DMSO", fontsize=18)
    plt.savefig(f"volcano_{treatment}.png")
    plt.close()
    # plt.show()


In [43]:
df = df[~df["Metadata_leiden"].isna()]

In [44]:
group = "Metadata_target"
group = "Metadata_Mode_of_Action"
#group = "Metadata_treatments"
group = "Metadata_leiden"

In [45]:
dmso_leidens = df.loc[df["Metadata_target"] == "DMSO", "Metadata_leiden"].unique()
dmso_leiden_dict = dict([(x, "DMSO") for x in dmso_leidens])
df["Metadata_leiden"] = df["Metadata_leiden"].replace(dmso_leiden_dict)

In [46]:
for group in ["Metadata_target", "Metadata_treatments", "Metadata_leiden"]:
    for treatment in df[group].unique():
        if treatment == "DMSO":
            continue
        df_small = df[df[group].isin([treatment, "DMSO"])].copy()
        summary_df = deseq(df_small, group, "DMSO")
        make_volcano(summary_df, treatment)


(200, 11746)
(200, 2)


/home/ec2-user/miniconda3/envs/bridge_env/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Fitting size factors...
... done in 0.07 seconds.



Using None as control genes, passed at DeseqDataSet initialization


Fitting dispersions...
... done in 32.38 seconds.

Fitting dispersion trend curve...
... done in 0.33 seconds.

Fitting MAP dispersions...
... done in 29.79 seconds.

Fitting LFCs...
... done in 2.45 seconds.

Calculating cook's distance...
... done in 0.18 seconds.

Replacing 7 outlier genes.

Fitting dispersions...
... done in 0.06 seconds.

Fitting MAP dispersions...
... done in 0.04 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Running Wald tests...
... done in 1.21 seconds.



Log2 fold change & Wald test p-value: Metadata_target lipid other  vs DMSO
                  baseMean  log2FoldChange     lfcSE      stat        pvalue  \
Bcin01g00010     24.830269       -2.251783  0.307282 -7.328071  2.334896e-13   
Bcin01g00020     15.499798       -1.929769  0.326937 -5.902577  3.578668e-09   
Bcin01g00030     81.027390       -2.625814  0.347506 -7.556165  4.151259e-14   
Bcin01g00040     27.295532       -2.583620  0.336919 -7.668359  1.742103e-14   
Bcin01g00050     14.464725       -1.373388  0.233373 -5.884955  3.981615e-09   
...                    ...             ...       ...       ...           ...   
ENSRNA049513861   0.044430        0.331456  4.959641  0.066831  9.467165e-01   
ENSRNA049513883   0.347546       -0.283505  0.516350 -0.549056  5.829667e-01   
ENSRNA049514038   0.215926       -0.612377  0.639439 -0.957678  3.382250e-01   
ENSRNA049514141   0.104523        0.203421  1.717707  0.118426  9.057304e-01   
ENSRNA049514255   0.220159       -0.450296  0

...

Log2 fold change & Wald test p-value: Metadata_treatments BCS-DG52208 vs DMSO
                   baseMean  log2FoldChange     lfcSE      stat    pvalue  \
Bcin01g00010      55.615284        0.215721  0.338990  0.636364  0.524539   
Bcin01g00020      38.664478        0.284204  0.398121  0.713863  0.475312   
Bcin01g00030     211.007950        0.230497  0.352927  0.653102  0.513691   
Bcin01g00040      75.002197        0.488184  0.371394  1.314464  0.188690   
Bcin01g00050      30.269388        0.446506  0.308964  1.445171  0.148410   
...                     ...             ...       ...       ...       ...   
ENSRNA049513861    0.121117       -0.284311  6.053867 -0.046964  0.962542   
ENSRNA049513883    0.670726        0.792050  0.944137  0.838914  0.401517   
ENSRNA049514038    0.394430       -1.192599  1.712483 -0.696415  0.486169   
ENSRNA049514141    0.218949       -0.568261  6.050495 -0.093920  0.925173   
ENSRNA049514255    0.362476       -0.031517  1.626626 -0.019376  0.984541  

/home/ec2-user/miniconda3/envs/bridge_env/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Fitting size factors...
... done in 0.03 seconds.



Using None as control genes, passed at DeseqDataSet initialization


Fitting dispersions...


KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

def volcano_comparison(treatments):
    """
    Create an interactive volcano plot for multiple treatments and save as HTML. 
    Attach meta data to individual genes.

    Parameters
    ----------
    treatments : list of strings
        List of treatment names.

    Returns
    -------
    fig : object
        The figure object of the volcano plot.
    """
    fig, ax = plt.subplots(figsize=(22, 12))
    target, treatments = treatments
    colour_dict = {
        "Under Threshold": "lightgrey",
        "Up-regulated": "cornflowerblue",
        "Down-regulated": "crimson"
    }

    markers = ['o', 'X', 'D', '^', 'v', 'p', '*']
    filtered_dataframes, overlapping_significant_genes = [], []
    

    for i, treatment in enumerate(treatments):
        table_path = f"{treatment}_differential_expression.csv"
        if not os.path.isfile(table_path):
            print(f"File {table_path} not found.")
            continue
        
        data_frame = pd.read_csv(table_path, index_col=0)
        
        # Check for duplicate indices
        duplicates = data_frame.index.duplicated(keep=False)

        # Create a new index with suffixes for duplicates
        if duplicates.any():
            # Create a dictionary to count occurrences of each index
            count_dict = {}
            new_index = []

            for idx in data_frame.index:
                if idx in count_dict:
                    count_dict[idx] += 1
                    new_index.append(f"{idx}_{count_dict[idx]}")
                else:
                    count_dict[idx] = 1
                    new_index.append(idx)

            # Assign the new unique index to the DataFrame
            data_frame.index = new_index
        
        data_frame["-Log10P"] = np.maximum(-np.log10(data_frame["padj"]), 0)
        data_frame["Type hit"] = "Under Threshold"
        data_frame.loc[(data_frame["log2FoldChange"].abs() > 1) & (data_frame["padj"] < 0.05), "Type hit"] = "Up-regulated"
        data_frame.rename(columns={
            "log2FoldChange": "Log(2) Fold-Change",
            "-Log10P": "-Log(10) of adjusted P-value"
        }, inplace=True)
        
        significant_data = data_frame[data_frame["Type hit"] != "Under Threshold"].sort_values("-Log(10) of adjusted P-value", ascending=False)
        significant_data["Treatment"] = treatment
        filtered_dataframes.append(significant_data)
        overlapping_significant_genes.append(set(significant_data.index))

        sns.scatterplot(data=data_frame, x="Log(2) Fold-Change", y="-Log(10) of adjusted P-value", alpha=0.6, color="gainsboro", 
                        edgecolor='none', marker=markers[i % len(markers)], ax=ax)
    if len(overlapping_significant_genes) < 2:
        print("No overlapping genes found.")
        return
    intersection_of_genes = set.intersection(*overlapping_significant_genes)
    n = min(len(intersection_of_genes), 10)
    if n <= 1:
        print("No overlapping genes found.")
        return
    colors = plt.get_cmap("tab10")(np.linspace(0, 1, n))
    
    overlapping_gene_data = pd.concat([df[df.index.isin(intersection_of_genes)] for df in filtered_dataframes], axis=0)

    if not overlapping_gene_data.empty:
        overlapping_gene_data["Top 10 overlapping\ngenes by significance"] = overlapping_gene_data.index
        top_genes = overlapping_gene_data.groupby("Top 10 overlapping\ngenes by significance")["-Log(10) of adjusted P-value"].max().nlargest(10).index
        overlapping_gene_data = overlapping_gene_data[overlapping_gene_data["Top 10 overlapping\ngenes by significance"].isin(top_genes)]

        # Sort the overlapping_gene_data by -Log(10) of adjusted P-value in descending order
        overlapping_gene_data = overlapping_gene_data.sort_values("-Log(10) of adjusted P-value", ascending=False)

        # Create a sorted list of unique genes based on their order in the sorted DataFrame
        sorted_genes = overlapping_gene_data["Top 10 overlapping\ngenes by significance"].unique()

        for i, gene in enumerate(sorted_genes):
            subset = overlapping_gene_data[overlapping_gene_data["Top 10 overlapping\ngenes by significance"] == gene]
            ax.plot(subset["Log(2) Fold-Change"], subset["-Log(10) of adjusted P-value"], ls="--", alpha=0.4, c=colors[i])
        
        #overlapping_gene_data["Top 10 overlapping\ngenes by significance"] = [gene_mapping[x].split(",")[0] for x in overlapping_gene_data["Top 10 overlapping\ngenes by significance"]]
        overlapping_gene_data = overlapping_gene_data.drop_duplicates(subset=["Top 10 overlapping\ngenes by significance", "Treatment"], keep="first")
        sns.scatterplot(
            data=overlapping_gene_data.reset_index(drop = True),
            x="Log(2) Fold-Change",
            y="-Log(10) of adjusted P-value",
            hue="Top 10 overlapping\ngenes by significance", 
            style="Treatment",
            markers=markers[:len(intersection_of_genes)],
            palette=colors, 
            s=100,
            edgecolor="white",
            linewidth=1, ax=ax
        )
    # Add threshold lines
    ax.axhline(y=-np.log10(0.05), lw=2, ls="--", c="grey")
    ax.axvline(x=1, lw=2, ls="--", c="grey")
    ax.axvline(x=-1, lw=2, ls="--", c="grey")
    
    ax.spines[['right', 'top']].set_visible(False)
    ax.grid(True, alpha=0.2)
    ax.tick_params(axis='both', which='major', labelsize=12)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

    ax.set_title(f"Volcano Plot of Target {target}", fontsize=16)
    plt.savefig(f"volcano_plot_{target}.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"volcano_plot_{target}.svg")
    plt.show()

In [ ]:
treatment_groupings = list(df.groupby(["Metadata_target"])["Metadata_treatments"])
treatment_groupings = [(x[0][0],list(x[1].unique())) for x in treatment_groupings if len(x[1]) > 1]
treatment_groupings = [x for x in treatment_groupings if len(x[1]) > 1]

In [ ]:
for grouping in treatment_groupings:
    print(grouping)
    volcano_comparison(grouping)